In [1]:
import pandas as pd
import scanpy as sc
import json

In [9]:
datasets = ['PBMC1', 'PBMC2', 'PBMC3', 'PBMC4']
data_dir = './data/'
for dataset in datasets:
    filtered_dataset_path = f"{data_dir}{dataset}/filtered/10X/"
    adata = sc.read_10x_mtx(filtered_dataset_path, var_names="gene_symbols", cache=False)
    filtered_barcodes = adata.obs.index.to_list()
    labels = pd.read_csv(f"{data_dir}{dataset}/celltypist/celltypist_labels.csv", index_col=0)
    labels.index = labels.index.str[:-2]
    labels = labels[labels.index.isin(filtered_barcodes)]
    counts = labels.value_counts().to_frame()
    counts.index.names = ['id']
    mapping = pd.read_csv(f"{data_dir}/{dataset}/celltypist/celltypist_mapping.csv", index_col=0)
    counts_mappings = counts.merge(mapping, on='id')
    counts_mappings.to_csv(f"{data_dir}/{dataset}/celltypist/celltypist_annotation_counts_filtered.csv")
    with open(f'{data_dir}/{dataset}/celltypist/nclusters.json', 'w') as fp:
        json.dump({'nclusters': counts_mappings[counts_mappings['count']>=50].shape[0]}, fp)